In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
# import custom libraries
import sys
sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
import hk_psql
import dash_bootstrap_components

import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots


ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"

%load_ext autoreload
%autoreload 2

conn = hk_psql.connect_psql('mimic')


In [ ]:
sel_icustay = [{'id_careunit': 'Medical Intensive Care Unit (MICU)', 'id_los_min': '24', 'id_los_max': '240'},
 {'id_careunit': 'Trauma SICU (TSICU)', 'id_los_min': '24', 'id_los_max': '240'},
 {'id_careunit': 'Surgical Intensive Care Unit (SICU)', 'id_los_min': '24', 'id_los_max': '240'}]

sel_icustay = [
    # {'id_careunit': 'ICU', 'id_los_min': '24', 'id_los_max': '1000'},
    # {'id_careunit': 'Cardiac Surgery', 'id_los_min': '0', 'id_los_max': '1000'},
]


filters_core = {
        'age':[20,91],
        'gender':['M','F'],
        'icustays':pd.DataFrame.from_dict(sel_icustay),
        'adm_types':[],
        'adm_locs':[],
        'dis_locs':[],

    }
# print(filters)
q_core = hk_psql.bq_core(filters_core)
# print(q_core)
df_filtered_core = hk_psql.query(q_core, conn)
hadm_ids_core = df_filtered_core['hadm_id'].unique().tolist()
print(df_filtered_core)
print('len after core: ',len(hadm_ids_core))

In [ ]:
# CABG related ICD codes - saves to saved_proc_cabg.csv

# ICD-9-CM codes
# link http://www.icd9data.com/2015/Volume3/35-39/36/default.htm?__hstc=93424706.1be12cb98a07e719a9f6f9964d70c68b.1646060204805.1649408353593.1649659786020.5&__hssc=93424706.18.1649659786020&__hsfp=1972159242#36.13

icd9=['3611', '3612', '3613', '3614', '3615', '3616', '3617', '3619']

# ICD-10-PCS codes

# eq to icd9 36.11
x3611=[
'021008W',
'021009W',
'02100AW',
'02100JW',
'02100KW',
'021048W',
'021049W',
'02104AW',
'02104JW',
'02104KW',]

# eq to icd9 36.11
x3611=[
'021008W',
'021009W',
'02100AW',
'02100JW',
'02100KW',
'021048W',
'021049W',
'02104AW',
'02104JW',
'02104KW',]

# eq to icd9 36.12
x3612=[
'021108W',
'021109W',
'02110AW',
'02110JW',
'02110KW',
'021148W',
'021149W',
'02114AW',
'02114JW',
'02114KW',]

# eq to icd9 36.13
x3613=[
'021208W',
'021209W',
'02120AW',
'02120JW',
'02120KW',
'021248W',
'021249W',
'02124AW',
'02124JW',
'02124KW',]

# eq to icd9 36.14
x3614=[
'021308W',
'021309W',
'02130AW',
'02130JW',
'02130KW',
'021348W',
'021349W',
'02134AW',
'02134JW',
'02134KW',]


# eq to icd9 36.15
x3615=[
'0210089', 
'021008C', 
'0210098', 
'0210099', 
'021009C', 
'02100A8', 
'02100A9', 
'02100AC', 
'02100J8', 
'02100J9', 
'02100JC', 
'02100K8', 
'02100K9', 
'02100KC', 
'02100Z8', 
'02100Z9', 
'02100ZC', 
'0210488', 
'0210489', 
'021048C', 
'0210498', 
'0210499', 
'021049C', 
'02104A8', 
'02104A9', 
'02104AC', 
'02104J8', 
'02104J9', 
'02104JC', 
'02104K8', 
'02104K9', 
'02104KC', 
'02104Z8', 
'02104Z9', 
'02104ZC',]



df_d_icd_proc = pd.read_csv(ADD_DATA+'d_icd_procedures.csv')

df_icd_CABG = df_d_icd_proc[df_d_icd_proc['icd_code'].isin([*icd9, *x3611, *x3612, *x3613, *x3614, *x3615])]
print([*icd9, *x3611, *x3612, *x3613, *x3614, *x3615])
df_icd_CABG.to_csv(ADD_DATA+'saved_proc_cabg.csv')
df_icd_CABG

In [ ]:
# **************** STEP 2: filter hosp codes (drg , icd)
'''	icd_code	icd_version	long_title	count
130	3613	9	(Aorto)coronary bypass of three coronary arteries	1074
258	021209W	10	Bypass Coronary Artery, Three Arteries from Aorta with Autologous Venous Tissue, Open Approach	520
'''

filters_codes = {
    'drgcodes': pd.DataFrame(), 
    'diagnoses_icd': pd.DataFrame(),
    'procedures_icd': pd.read_csv(ADD_DATA+'saved_proc_cabg.csv'),
}


q = hk_psql.bq_codes(filters_codes, hadm_ids_core)
# q = hk_psql.bq_codes(filters_codes, [])
print('sdfds')
if len(filters_codes['procedures_icd'])+len(filters_codes['diagnoses_icd'])+len(filters_codes['drgcodes'])==0:
    # if no code is specified, just ignore query
    df_filtered_codes = df_filtered_core
else:
    df_filtered_codes = hk_psql.query(q, conn)
# print(df_filtered_codes)
# df_filtered_core.to_csv(ADD_DATA+'temp.csv')
hadm_ids_core_code = df_filtered_codes['hadm_id'].unique().tolist()
print('len after code: ', len(hadm_ids_core_code))



In [ ]:
# **************** STEP 3: Hosp filters -> hadm_ids
lab_table = [
    # {'id_item': 50813, 'id_label': 'Lactate', 'id_fluid': 'Blood', 'id_cat': 'Blood Gas', 'id_range': '0.5-2.0', 'id_unit': 'mmol/L', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50889, 'id_label': 'C-Reactive Protein', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-5.0', 'id_unit': 'mg/L', 'id_condition': 'X>0', 'id_counts': 10}, 
{'id_item': 51003, 'id_label': 'Troponin T', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-0.01', 'id_unit': 'ng/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50963, 'id_label': 'NTproBNP', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-177.0', 'id_unit': 'pg/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50924, 'id_label': 'Ferritin', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '13.0-150.0', 'id_unit': 'ng/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50909, 'id_label': 'Cortisol', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '2.0-20.0', 'id_unit': 'ug/dL', 'id_condition': 'X>0', 'id_counts': 1},
]       


filters_hosp = {
    'labevents': pd.DataFrame.from_dict(lab_table),
}
filters_hosp = {
    'labevents': pd.DataFrame(),
}


q_hosp = hk_psql.bq_hosp(hadm_ids_core_code, filters_hosp)

df_filtered_hosp = hk_psql.query(q_hosp, conn)
print(df_filtered_hosp)
df_filtered_core = pd.merge(df_filtered_hosp[['hadm_id']], df_filtered_core[['subject_id','hadm_id','anchor_age','gender','careunit','icu_los','stay_id']], how='inner',on=['hadm_id'])
df_filtered = pd.merge(df_filtered_hosp, df_filtered_core[['subject_id','hadm_id','anchor_age','gender','careunit','icu_los','stay_id']], how='inner',on=['hadm_id'])

print(df_filtered_core)
# print(df_filtered_hosp)        
print(df_filtered)

hadm_ids = df_filtered_core['hadm_id'].unique().tolist()
stay_ids = df_filtered_core['stay_id'].unique().tolist()
sub_ids = df_filtered_core['subject_id'].unique().tolist()

print('len after hosp: ',len(hadm_ids))

In [ ]:
cohort_name = 'cabg'
hk_psql.save_cohort(hadm_ids, cohort_name)

In [ ]:
# load a cohort
cohort_name = 'cabg'
schema_core = cohort_name+'_mimic_core'
df_overview = hk_psql.summary(schema_core, conn)
hadm_ids = df_overview['hadm_id'].unique()

q=f"""
    select * from mimic_core.patients t1
    inner join {schema_core}.admissions t2
    on t1.subject_id = t2.subject_id
    where t2.hadm_id in ({str(hadm_ids.tolist())[1:-1]})
    
    """
# print(q)

df_patients = hk_psql.query(q, conn)

q=f"""
        select * from {schema_core}.transfers         
        """
# print(q)

df_transfers = hk_psql.query(q, conn)

q=f"""
        select * from {schema_core}.admissions         
        """
# print(q)

df_admissions = hk_psql.query(q, conn)

q=f"""
        select * from {cohort_name+'_mimic_icu'}.icustays         
        """
# print(q)

df_icustay = hk_psql.query(q, conn)

# LAB EVENTS
q=f"""
select * from {cohort_name+'_mimic_hosp'}.labevents         
"""

df_le = hk_psql.query(q, conn)

# icd_procedure code
q=f"""
select * from {cohort_name+'_mimic_hosp'}.procedures_icd         
"""
df_icd_proc = hk_psql.query(q, conn)

# icd_diagnoses code
q=f"""
select * from {cohort_name+'_mimic_hosp'}.diagnoses_icd         
"""
df_icd_diag = hk_psql.query(q, conn)


# CHART EVENTS
q=f"""
select * from {cohort_name+'_mimic_icu'}.chartevents         
"""

df_che = hk_psql.query(q, conn)

# PROCEDURE EVENTS
q=f"""
select * from {cohort_name+'_mimic_icu'}.procedureevents         
"""

df_pe = hk_psql.query(q, conn)

df_patients = df_patients.loc[:,~df_patients.columns.duplicated()]
print(df_patients['subject_id'])
print(df_transfers['subject_id'])


cols_to_use = [x for x in df_transfers.columns.tolist() if x not in df_patients.columns.tolist()]
cols_to_use.append('subject_id')
print(cols_to_use)
df_demo = pd.merge(df_patients, df_transfers[cols_to_use], how='inner',  on='subject_id')

print(df_demo)
df_le_main = df_le.copy()

In [347]:

df_temp = df_transfers.groupby(['hadm_id','careunit']).size().reset_index(name='count').sort_values(by=['hadm_id','careunit','count',],ascending=False)

df_temp = df_temp.groupby(['careunit','count']).size().reset_index(name='size').sort_values(by='size',ascending=False).iloc[:]
df_temp
df_temp['size'] = df_temp['size']/len(hadm_ids) * 100


# FIGURE: frequency of multiple careunits
px.histogram(df_temp, y = 'size', x='careunit', color='count',
#labels={'size':'%'},
 barmode='group',
 title = "frequency of multiple careunits")



In [346]:
# FIGURE: top-10 frequent care units

px.histogram(df_temp, y = 'size', x='careunit',
#labels={'size':'%'},
 barmode='group',
 title = "frequency of careunits")

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['subject_id', 'hadm_id', 'transfer_id', 'eventtype', 'label', 'intime', 'outtime', 'detail', 'icd_code', 'icd_version', 'admittime', 'id', 'color', 'delta'] but received: careunit

In [ ]:
from datetime import datetime, timedelta

date_time_str = '01/01/00 01:55:19'

date_origin = datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')

df2_transfers = df_transfers
df2_transfers['detail'] = df2_transfers['careunit']
df2_transfers = df2_transfers.rename(columns={'careunit':'label'})
# drop outtime=nan
df2_transfers = df2_transfers.dropna(subset='outtime')
df2_transfers.dtypes


In [ ]:
# icd-proc codes CABG 

from datetime import datetime, timedelta
df_d_icd_proc = pd.read_csv(ADD_DATA+'d_icd_procedures.csv')



df_icd_CABG = pd.read_csv(ADD_DATA+'saved_proc_cabg.csv')


temp = df_icd_CABG['icd_code'].tolist()
df_icd_CABG = df_icd_proc.query(f"icd_code.isin({temp})")
df_icd_CABG = pd.merge(df_icd_CABG, df_d_icd_proc,on=['icd_code', 'icd_version'])

df_icd_CABG['intime'] = df_icd_CABG ['chartdate'] 
df_icd_CABG['outtime'] =  df_icd_CABG ['chartdate'] + timedelta(days=1)

df_icd_CABG['detail'] = df_icd_CABG['icd_code'] + '/' +df_icd_CABG['long_title']



df_icd_CABG['label']='CABG'

df_icd_CABG = df_icd_CABG[['hadm_id','label','intime','outtime','detail','icd_code','icd_version']]

print(df_icd_CABG.dtypes)

In [345]:
# icd-diag codes containing "myocardial infarction"  except "old"

df_d_icd_diag = pd.read_csv(ADD_DATA+'d_icd_diagnoses.csv')

df2_icd_diag = pd.merge(df_icd_diag, df_d_icd_diag,on=['icd_code', 'icd_version'])

df_icd_MI = df2_icd_diag.query(f"long_title.str.contains('myocardial infarction',case=False)")

df_icd_MI['label'] = 'MI'
df_icd_MI['detail'] = df_icd_MI['icd_code'] + '/' +df_icd_MI['long_title']
df_icd_MI['intime'] = date_origin - timedelta(days=3)
df_icd_MI['outtime'] = date_origin - timedelta(days=2)
df_icd_MI = df_icd_MI[['hadm_id','label','detail','intime','outtime']]
print(df_icd_MI.dtypes)

# FIGURE: plot most frequent MI-related codes in the cohort
temp = df_icd_MI.groupby('detail').size().reset_index(name='size').sort_values(by='size',ascending=False)
fig = px.bar(temp,x='detail', y='size', title = 'most frequent MI-related codes')
fig.show()
fig.write_html(ADD_DATA+'MI-related codes.html')
print(len(df_icd_MI))


hadm_id             int64
label              object
detail             object
intime     datetime64[ns]
outtime    datetime64[ns]
dtype: object


1918


In [348]:
# "cath lab" as outcome from icu.procedures

# 225430	Cardiac Cath
# 229577	Cath Lab (Received)
# 229578	Cath Lab (Sent)
list_itemids = [
225430,
229577,
229578,
]

dict_cathlab = {
225430	: 'Cardiac Cath',
229577	: 'Cath Lab (Received)',
229578	: 'Cath Lab (Sent)',
}

df2_pe = df_pe.query(f"itemid.isin({list_itemids})")

df2_pe = df2_pe[['hadm_id','starttime','endtime','itemid']]
df2_pe = df2_pe.rename(columns={'starttime':'intime','endtime':'outtime','itemid':'label'})
df2_pe['outtime'] = df2_pe['outtime']+timedelta(days=1)

df2_pe = df2_pe.replace({'label': dict_cathlab})

df2_pe['detail'] = df2_pe['label']

print(df2_pe.dtypes)
print(df2_pe)


hadm_id             int64
intime     datetime64[ns]
outtime    datetime64[ns]
label              object
detail             object
dtype: object
        hadm_id              intime             outtime                label  \
203    23891344 2118-09-29 17:00:00 2118-09-30 17:01:00         Cardiac Cath   
1223   23531962 2128-08-22 08:45:00 2128-08-23 08:46:00         Cardiac Cath   
1265   22848187 2125-04-27 15:00:00 2125-04-28 15:01:00         Cardiac Cath   
1509   25849202 2185-04-08 11:30:00 2185-04-09 11:31:00         Cardiac Cath   
1546   27973178 2180-05-16 14:30:00 2180-05-17 14:31:00         Cardiac Cath   
...         ...                 ...                 ...                  ...   
87119  28948855 2155-11-02 20:20:00 2155-11-03 20:21:00  Cath Lab (Received)   
87142  28948855 2155-11-09 16:00:00 2155-11-10 16:01:00         Cardiac Cath   
87452  21224034 2176-04-05 15:30:00 2176-04-06 15:31:00      Cath Lab (Sent)   
87453  21224034 2176-04-05 15:42:00 2176-04-06 15:43:00 

In [349]:
df_concat = pd.concat([df2_transfers, df_icd_CABG, df_icd_MI, df2_pe])

df_concat.dtypes

subject_id            float64
hadm_id                 int64
transfer_id           float64
eventtype              object
label                  object
intime         datetime64[ns]
outtime        datetime64[ns]
detail                 object
icd_code               object
icd_version           float64
dtype: object

In [350]:
df_concat['label'].value_counts()
df_concat = pd.merge(df_concat,df_admissions[['hadm_id','admittime']], on='hadm_id')


In [351]:


# select a subset
#df_concat=df_concat.query(f"careunit=='Cardiac Surgery' or careunit.str.endswith('(CVICU)') or careunit=='CABG' or careunit.isin({list_itemids}) ")

# select a subset that has Cath lab
sub_hadm_ids = df_concat[df_concat['label'].str.contains('Cath').fillna(False)]['hadm_id'].unique().tolist()
df2_concat = df_concat[df_concat['hadm_id'].isin(sub_hadm_ids)]

df2_concat ['intime'] = (df2_concat ['intime'] - df2_concat ['admittime']) + date_origin
df2_concat ['outtime'] = (df2_concat ['outtime'] - df2_concat ['admittime']) +date_origin
q_MI = df2_concat['label']=='MI'
df2_concat.loc [q_MI, 'intime'] = date_origin - timedelta(days=3)
df2_concat.loc [q_MI, 'outtime'] = date_origin - timedelta(days=2)
df2_concat = df2_concat.sort_values(['hadm_id','intime']).iloc[:] 




In [352]:
print(f"total CABG patients: {len(df_transfers['hadm_id'].unique())}")
print(f"total number with an outcome: {len(df2_concat['hadm_id'].unique())}")

df2_concat['label'].value_counts()

total CABG patients: 5580
total number with an outcome: 117


Cardiac Vascular Intensive Care Unit (CVICU)        220
CABG                                                212
Cardiac Surgery                                     120
Coronary Care Unit (CCU)                             88
Cardiac Cath                                         86
MI                                                   68
Medicine/Cardiology                                  63
PACU                                                 62
Emergency Department                                 54
Cath Lab (Sent)                                      39
Cardiology Surgery Intermediate                      38
Cath Lab (Received)                                  22
Medical Intensive Care Unit (MICU)                   15
Medicine                                              7
Medicine/Cardiology Intermediate                      5
Vascular                                              5
Trauma SICU (TSICU)                                   4
Neurology                                       

In [353]:

# assign hadm_id to a unique list [1: n_adm]
L1 = df2_concat['hadm_id'].unique().tolist()
L2 = list(range(1,len(hadm_ids)+1))
mapper=dict(zip(L1, L2))
df2_concat['id'] = df2_concat['hadm_id']
df2_concat = df2_concat.replace({"id": mapper})

#df2_concat['id'] =df2_concat['id']
list_temp = [*list(dict_cathlab.values()),'CABG']
q = df2_concat['label'].isin(list_temp)
df2_concat.loc[q,'id'] = df2_concat.loc[q,'id'] + 0.2


df2_concat = df2_concat[df2_concat.label != 'Emergency Department']
df2_concat = df2_concat.dropna(subset='label')

In [354]:
# define colors
df2_concat['color'] = 'Other units'

q = df2_concat['label'].str.contains('ICU').fillna(False)
df2_concat.loc[q,'color']='ICU'

q = df2_concat['label'].str.contains('CCU').fillna(False)
df2_concat.loc[q,'color']='ICU'

q = df2_concat['label'].str.contains('Surgery').fillna(False)
df2_concat.loc[q,'color']='Surgery'

q = df2_concat['label'].isin(['CABG']).fillna(False)
df2_concat.loc[q,'color']='CABG'

q = df2_concat['label'].str.contains('Cath').fillna(False)
df2_concat.loc[q,'color']='Outcome'

q = df2_concat['label'].isin(['MI']).fillna(False)
df2_concat.loc[q,'color']='MI'

df2_concat

,subject_id,hadm_id,transfer_id,eventtype,label,intime,outtime,detail,icd_code,icd_version,admittime,id,color
18103,NaN,20167242,NaN,NaN,MI,1999-12-29 01:55:19,1999-12-30 01:55:19,I214 /Non-ST elevation (NSTEMI) myocardial i...,NaN,NaN,2144-10-27 00:11:00,1.0,MI
18101,NaN,20167242,NaN,NaN,CABG,2000-01-02 01:44:19,2000-01-03 01:44:19,"02100Z9/Bypass Coronary Artery, One Artery fro...",02100Z9,10.0,2144-10-27 00:11:00,1.2,CABG
18102,NaN,20167242,NaN,NaN,CABG,2000-01-02 01:44:19,2000-01-03 01:44:19,"02120KW/Bypass Coronary Artery, Three Arteries...",02120KW,10.0,2144-10-27 00:11:00,1.2,CABG
18099,18871076.0,20167242,30417835.0,admit,Cardiac Vascular Intensive Care Unit (CVICU),2000-01-02 01:57:44,2000-01-05 00:08:59,Cardiac Vascular Intensive Care Unit (CVICU),NaN,NaN,2144-10-27 00:11:00,1.0,ICU
18104,NaN,20167242,NaN,NaN,Cath Lab (Sent),2000-01-02 12:37:19,2000-01-03 12:38:19,Cath Lab (Sent),NaN,NaN,2144-10-27 00:11:00,1.2,Outcome
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3230,17879591.0,29871031,32124959.0,transfer,Medicine/Cardiology,2000-01-03 01:19:07,2000-01-05 21:58:58,Medicine/Cardiology,NaN,NaN,2127-08-04 15:05:00,117.0,Other units
3231,17879591.0,29871031,34465756.0,transfer,Medicine/Cardiology,2000-01-05 21:58:58,2000-01-08 20:32:30,Medicine/Cardiology,NaN,NaN,2127-08-04 15:05:00,117.0,Other units
3234,NaN,29871031,NaN,NaN,CABG,2000-01-08 10:50:19,2000-01-09 10:50:19,3613/(Aorto)coronary bypass of three coronary ...,3613,9.0,2127-08-04 15:05:00,117.2,CABG
3232,17879591.0,29871031,36704045.0,transfer,Cardiac Vascular Intensive Care Unit (CVICU),2000-01-08 20:32:30,2000-01-10 08:15:31,Cardiac Vascular Intensive Care Unit (CVICU),NaN,NaN,2127-08-04 15:05:00,117.0,ICU


In [355]:



fig = px.timeline(df2_concat.iloc[:], y='id', x_start='intime', x_end = 'outtime', color='color',
        hover_data=['hadm_id','detail'])
fig.update_traces(width=0.2)
fig.show()
fig.write_html(ADD_DATA+'CABG timeline.html')



In [403]:
df=pd.read_csv(ADD_DATA+'d_labitems.csv').query("label.str.contains('lactate',case=False)").drop_duplicates('itemid')
df

,itemid,label,valueuom,ref_range_lower,ref_range_upper,fluid,category
729,50813,Lactate,mmol/L,0.5,2.0,Blood,Blood Gas
730,50954,Lactate Dehydrogenase (LD),IU/L,94.0,250.0,Blood,Chemistry
734,50843,"Lactate Dehydrogenase, Ascites",IU/L,NaN,NaN,Ascites,Chemistry
735,51795,"Lactate Dehydrogenase, CSF",IU/L,NaN,NaN,Cerebrospinal Fluid,Chemistry
736,51054,"Lactate Dehydrogenase, Pleural",IU/L,NaN,NaN,Pleural,Chemistry


In [413]:
new_id = 27497725
dict_le_bio = {
51003:	'Troponin T',
50910:	'Creatine Kinase (CK)',
50911:	'Creatine Kinase, MB Isoenzyme',
50813:	'Lactate',
}

dict_che_bio = {
#227429:	'Troponin-T',
220045:	'Heart Rate',	
}
#dict_che_bio = {}
q_le = (df_le['hadm_id']==new_id) & ( df_le['itemid'].isin(list(dict_le_bio.keys())) )
df2_le = df_le[q_le]
df2_le['label'] = df2_le['itemid'].replace(dict_le_bio)

q_che = (df_che['hadm_id']==new_id) & ( df_che['itemid'].isin(list(dict_che_bio.keys())) )
df2_che = df_che[q_che]
df2_che['label'] = df2_che['itemid'].replace(dict_che_bio)

df2_che

df_bio = pd.concat([df2_le[['hadm_id','itemid','label','charttime', 'valuenum']],
    df2_che[['hadm_id','itemid','label','charttime', 'valuenum']],]
    )
df_bio = pd.merge(df_bio,df_admissions[['hadm_id','admittime']], on='hadm_id')
df_bio ['time'] = (df_bio ['charttime'] - df_bio ['admittime']) + date_origin

df_bio = df_bio.sort_values(by=['itemid','time'])
#px.line(df_bio,x='time',y='valuenum',color='label')
# df=pd.read_csv(ADD_DATA+'d_labitems.csv').query("label.str.contains('creatin',case=False)").drop_duplicates('itemid')
# df

df_bio['time'] = (df_bio['time'] - date_origin) / np.timedelta64(1,'h')

list_bio = df_bio['itemid'].unique().tolist()

fig = make_subplots(rows=len(list_bio)+1, cols=1,
            shared_xaxes=True,
            subplot_titles=[*df_bio['label'].unique(),'Time line']
            )

for i,itemid in enumerate(list_bio):
    df_temp = df_bio[df_bio['itemid']==itemid]
    fig.add_trace(
        go.Scatter(x=df_temp['time'], y=df_temp['valuenum'],
                    mode='lines+markers',
                    name=df_temp['label'].iloc[0],
        ),
        row=i+1, col=1
    )






In [414]:


df_temp = df2_concat[df2_concat['hadm_id']==new_id]
df_temp['intime'] = (df_temp['intime'] - date_origin) / np.timedelta64(1,'h')
df_temp['outtime'] = (df_temp['outtime'] - date_origin) / np.timedelta64(1,'h')
#df_temp['id']= 'Events'

q_cu = df_temp['id']%1==0
q_events = ~q_cu




df_temp.loc[q_cu,'id'] = 'Care units'
df_temp.loc[q_events,'id'] = 'Events'

fig_temp = hk_psql.plot_timeline(df_temp, x_start="intime", x_end="outtime", y='id', color='color',hover_data=['hadm_id','detail'])
fig_temp.update_traces(width=0.2)
  
for trace in fig_temp.data:
    #print(trace)
    fig.add_trace(trace, row=len(list_bio)+1, col=1)



In [415]:
fig.update_layout(title_text = f"id: {new_id}")
fig.show()


fig.write_html(ADD_DATA+f'fig_timeline_id{new_id}.html')


In [ ]:
df_che.dtypes
print(df_che[df_che['itemid'].isin(list_itemids)])
df_che

In [ ]:
df_che_main = df_che.copy()

In [ ]:
df_che[df_che['itemid']==228445]


In [ ]:
# read desired itemids from csv files

ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"
df= pd.read_csv(ADD_DATA+'d_lookup_le.csv')
le_itemdids = df['itemid']


df= pd.read_csv(ADD_DATA+'d_lookup_che.csv')
che_itemdids = df['itemid']


grid='1H'


In [ ]:
df_le = df_le_main
df_che = df_che_main
print(len(df_demo['hadm_id'].unique()))
print(len(df_le['hadm_id'].unique()))
print(len(df_icustay['stay_id'].unique()))

icustay_ids = len(df_icustay['stay_id'].unique())

df_le = df_le[df_le['itemid'].isin(le_itemdids)]
df_che = df_che[df_che['itemid'].isin(che_itemdids)]

print(df_le['itemid'].value_counts())


df_le = pd.merge(df_le[['hadm_id','itemid','charttime','valuenum']],
         df_icustay, on='hadm_id',how='inner')

df_che = pd.merge(df_che[['hadm_id','itemid','charttime','valuenum']],
         df_icustay, on='hadm_id',how='inner')

In [ ]:
# filtering icu stays, reset charttime to start from zero (icu admission)

df_le['temp1'] = round((df_le['charttime'] - df_le['intime']) / np.timedelta64(1, 's')/3600,2)
df_le['temp2'] = round((df_le['outtime'] - df_le['charttime']) / np.timedelta64(1, 's')/3600,2)
df_le = df_le[ (df_le['temp1']>0) & (df_le['temp2']>0) ]
df_le['charttime'] = df_le['temp1']
df_le = df_le.sort_values(['stay_id','itemid', 'charttime'])

df_che['temp1'] = round((df_che['charttime'] - df_che['intime']) / np.timedelta64(1, 's')/3600,2)
df_che['temp2'] = round((df_che['outtime'] - df_che['charttime']) / np.timedelta64(1, 's')/3600,2)
df_che = df_che[ (df_che['temp1']>0) & (df_che['temp2']>0) ]
df_che['charttime'] = df_che['temp1']
df_che = df_che.sort_values(['stay_id','itemid', 'charttime'])




In [ ]:
#convert charttime(int) to hours, and resample and forward fill imputation

from datetime import datetime


df_le = df_le[['stay_id','itemid','charttime','valuenum']][:]
df1=df_le
df_le['charttime'] = datetime(2000,1,1,0,0,0) + df_le.charttime.astype('timedelta64[h]')
df_le = df_le.set_index('charttime')
# print(df_le.index)
df_le= df_le.groupby(['stay_id','itemid']).resample(grid)['valuenum'].last().reset_index(drop=False)
df_le['valuenum']= df_le.groupby(['stay_id','itemid'])['valuenum'].fillna(method='ffill')
df_le = df_le.sort_values(['stay_id','itemid', 'charttime'])



df_che = df_che[['stay_id','itemid','charttime','valuenum']][:]
df1=df_che
df_che['charttime'] = datetime(2000,1,1,0,0,0) + df_che.charttime.astype('timedelta64[h]')
df_che = df_che.set_index('charttime')
# print(df_che.index)
df_che= df_che.groupby(['stay_id','itemid']).resample(grid)['valuenum'].last().reset_index(drop=False)
df_che['valuenum']= df_che.groupby(['stay_id','itemid'])['valuenum'].fillna(method='ffill')
df_che = df_che.sort_values(['stay_id','itemid', 'charttime'])


In [ ]:
# pivot dataframes so that each measurement is a column
# concatenate df_le and df_che

df_lee = df_le.set_index(['charttime','stay_id'])
df_lee = df_lee.pivot(columns = 'itemid', values='valuenum')

df_chee = df_che.set_index(['charttime','stay_id'])
df_chee = df_chee.pivot(columns = 'itemid', values='valuenum')

df = pd.concat([df_lee, df_chee], axis=1)
# df = df_chee


df = df.reset_index(drop=False)
df = df.sort_values(['stay_id', 'charttime'])


print(df.isnull().sum())
print(len(df))


print(df.dropna().isnull().sum())
print(len(df.dropna()))

df = df.dropna(subset=[51003])


In [ ]:
df
print(df['stay_id'].value_counts())
sample_id = df['stay_id'].value_counts().index[0]
print(sample_id)

In [ ]:
# plot before normalization

stay_ids = df_icustay['stay_id'].unique().tolist()
# print(df['stay_id'].value_counts())


df_temp = df[df['stay_id']==sample_id ]
fig = px.line(df_temp, x='charttime', y=[220045, 220210, 51003], markers=True)
fig.show()


# df_temp = df_le[df_le['stay_id']==39132344    ]
# fig = px.line(df_temp, x='charttime', y='valuenum', color='itemid', markers=True)
# fig.show()


# df_temp = df_che[df_che['stay_id']==39132344    ]
# fig = px.line(df_temp, x='charttime', y='valuenum', color='itemid', markers=True)
# fig.show()

In [ ]:
df_temp

In [ ]:
# normalizing the data

from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()
# transform data
df_temp = df_temp.set_index(['charttime','stay_id'])

df_temp = pd.DataFrame(scaler.fit_transform(df_temp), index=df_temp.index, columns=df_temp.columns)
df_temp = df_temp.reset_index(drop=False)


In [ ]:
# plot after normalization

print(df_temp.columns)
fig = px.line(df_temp, x='charttime', y=[       220045,    220210,  ], markers=True)
fig.show()


In [ ]:
from sklearn.model_selection import GroupShuffleSplit
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 7)
split = splitter.split(df, groups=df['stay_id'])
train_inds, test_inds = next(split)

df_train = df.iloc[train_inds]
df_test = df.iloc[test_inds]

print(len(df['stay_id'].unique()))
print(len(df_train['stay_id'].unique()))
print(len(df_test['stay_id'].unique()))

In [ ]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

n_past = 3  # * grid
n_future = 2 # * grid
n_features = 2




# convert train dataframe to numpy
stay_ids = df_train['stay_id'].unique()
list_X = []
list_y = []
for i in stay_ids:
  df_temp = df_train[df_train['stay_id']==i].drop(columns=['stay_id','charttime']).to_numpy()
  X_train, y_train = split_series(df_temp ,n_past, n_future)
  
  list_X.append(X_train)
  list_y.append(y_train)

list_X = [i for i in list_X if i.ndim==3]
list_y = [i for i in list_y if i.ndim==3]

X_train=np.concatenate(list_X, axis = 0)
y_train=np.concatenate(list_y, axis = 0)



# convert test dataframe to numpy
stay_ids = df_test['stay_id'].unique()
list_X = []
list_y = []
for i in stay_ids:
  df_temp = df_test[df_test['stay_id']==i].drop(columns=['stay_id','charttime']).to_numpy()
  X_test, y_test = split_series(df_temp ,n_past, n_future)

  list_X.append(X_test)
  list_y.append(y_test)

list_X = [i for i in list_X if i.ndim==3]
list_y = [i for i in list_y if i.ndim==3]

X_test=np.concatenate(list_X, axis = 0)
y_test=np.concatenate(list_y, axis = 0)


print(f""" 
X_train.shape = {X_train.shape}
y_train.shape = {y_train.shape}
X_test.shape = {X_test.shape}
y_test.shape = {y_test.shape}
 """)


In [ ]:

import tensorflow as tf
import os
n_features=X_train.shape[-1]
n_past=X_train.shape[-2]
n_future=y_train.shape[-1]

print(X_train.shape, X_test.shape)

In [ ]:
encoder = tf.keras.models.Sequential([
    # tf.keras.layers.LSTM(100, return_state=True, input_shape = (n_past,n_features)),
    tf.keras.layers.Flatten(input_shape=[3, 2]),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu"),
])

In [ ]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])


In [ ]:
print(history_e1d1.history['loss'])
plt.plot(history_e1d1.history['loss'])
plt.plot(history_e1d1.history['val_loss'])
plt.title("E1D1 Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Valid'])
plt.show()

In [ ]:
pred1_e1d1=model_e1d1.predict(X_test)
# pred1_e2d2=model_e2d2.predict(X_test)

pred_e1d1=model_e1d1.predict(X_train)
# pred_e2d2=model_e2d2.predict(X_train)

In [ ]:
print(pred1_e1d1.shape)
print(y_test.shape)
temp_test = y_test.reshape(-1,2)
temp_pred = pred1_e1d1.reshape(-1,2)


print(temp_pred.shape)
df_temp = pd.DataFrame(temp_pred[:1000,:],columns=[       220045,    220210,  ])
fig = px.line(df_temp, y=[       220045,    220210,  ], markers=True)

df_temp = pd.DataFrame(temp_test[:1000,:],columns=[       220045,    220210,  ])
fig = px.line(df_temp, y=[       220045,    220210,  ], markers=True)


fig.show()

In [ ]:
normal_df = df
RANDOM_SEED = 1374

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
  normal_df,
  test_size=0.15,
  random_state=RANDOM_SEED
)
val_df, test_df = train_test_split(
  val_df,
  test_size=0.33,
  random_state=RANDOM_SEED
)

In [ ]:
# Data preprocessing
def create_dataset(df):
  sequences = df.astype(np.float32).to_numpy().tolist()
  dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]
  n_seq, seq_len, n_features = torch.stack(dataset).shape
  return dataset, seq_len, n_features


train_dataset, seq_len, n_features = create_dataset(train_df)
val_dataset, _, _ = create_dataset(val_df)
test_normal_dataset, _, _ = create_dataset(test_df)
test_anomaly_dataset, _, _ = create_dataset(anomaly_df)

In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(nn.Module):
  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()
    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
    self.rnn1 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )
  def forward(self, x):
    x = x.reshape((1, self.seq_len, self.n_features))
    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)
    return hidden_n.reshape((self.n_features, self.embedding_dim))


class Decoder(nn.Module):
  def __init__(self, seq_len, input_dim=64, n_features=1):
    super(Decoder, self).__init__()
    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 2 * input_dim, n_features
    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )
    self.rnn2 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.output_layer = nn.Linear(self.hidden_dim, n_features)
  def forward(self, x):
    x = x.repeat(self.seq_len, self.n_features)
    x = x.reshape((self.n_features, self.seq_len, self.input_dim))
    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)
    x = x.reshape((self.seq_len, self.hidden_dim))
    return self.output_layer(x)

class RecurrentAutoencoder(nn.Module):
  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()
    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)
  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x



In [ ]:
import copy

def train_model(model, train_dataset, val_dataset, n_epochs):
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  criterion = nn.L1Loss(reduction='sum').to(device)
  history = dict(train=[], val=[])
  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 10000.0
  for epoch in range(1, n_epochs + 1):
    model = model.train()
    train_losses = []
    for seq_true in train_dataset:
      optimizer.zero_grad()
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)
      loss = criterion(seq_pred, seq_true)
      loss.backward()
      optimizer.step()
      train_losses.append(loss.item())
    val_losses = []
    model = model.eval()
    with torch.no_grad():
      for seq_true in val_dataset:
        seq_true = seq_true.to(device)
        seq_pred = model(seq_true)
        loss = criterion(seq_pred, seq_true)
        val_losses.append(loss.item())
    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)
    history['train'].append(train_loss)
    history['val'].append(val_loss)
    if val_loss < best_loss:
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
    print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
  model.load_state_dict(best_model_wts)
  return model.eval(), history

In [ ]:
model = RecurrentAutoencoder(seq_len, n_features, 128)
model = model.to(device)

model, history = train_model(
  model,
  train_dataset,
  val_dataset,
  n_epochs=150
)